In [113]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import statistics # статистические операции в (5)
import datetime # работа с датой в (21)
import itertools # формирование перестановок (27)

# Загрузка данных и предобработка

In [114]:
# рабочие переменные и словари
answers = {} # словарь для ответов
seasons = {'Зима': [1, 2, 12], 'Весна': [3, 4, 5], 'Лето': [6, 7, 8], 'Осень': [9, 10, 11]} # словарь сезонов
months = {1: 'Январь', 2: 'Февраль', 12: 'Декабрь',\
          3: 'Март', 4: 'Апрель', 5: 'Май',\
          6: 'Июнь', 7: 'Июль', 8: 'Август',\
          9: 'Сентябрь', 10: 'Октябрь', 11: 'Ноябрь'} # словарь месяцев

In [115]:
films = pd.read_csv('movie_bd_v5.csv') # загрузить входной массив данных

In [116]:
# предобработка входного датасета
films['profit'] = films['revenue'] - films['budget'] # прибыль фильма
films['genres'] = films['genres'].str.split('|') # парсинг жанров
films['director'] = films['director'].str.split('|') # парсинг режисёров
films['cast'] = films['cast'].str.split('|') # парсинг актёров
films['production_companies'] = films['production_companies'].str.split('|') # парсинг киностудий
films['release_date'] = pd.to_datetime(films['release_date'])
films['release_month'] = [film_date.month for film_date in films['release_date']]

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [117]:
# В случае, если макс.бюджет сразу у нескольких фильмов - берём первый, чтобы формально соблюсти условие задачи
answers['1'] = films.original_title[films.budget == films.budget.max()].iloc[0] # фильм с максимальным бюджетом
print(answers['1']) # +

Pirates of the Caribbean: On Stranger Tides


# 2. Какой из фильмов самый длительный (в минутах)?

In [118]:
answers['2'] = films.original_title[films.runtime == films.runtime.max()].iloc[0]
print(answers['2']) # +

Gods and Generals


# 3. Какой из фильмов самый короткий (в минутах)?

In [119]:
answers['3'] = films.original_title[films.runtime==films.runtime.min()].iloc[0]
print(answers['3']) # +

Winnie the Pooh


# 4. Какова средняя длительность фильмов?

In [120]:
answers['4'] = int(round(films.runtime.mean(), 0)) # округление до целого с приведением к int (по условиям ответов)
print(answers['4']) # +

110


# 5. Каково медианное значение длительности фильмов?

In [121]:
answers['5'] = int(round(statistics.median(films.runtime), 0)) # велосипед не изобретаем
print(answers['5']) # +

107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [122]:
answers['6'] = films.original_title[films.profit==films.profit.max()].iloc[0]
print(answers['6']) # +

Avatar


# 7. Какой фильм самый убыточный?

In [123]:
answers['7'] = films.original_title[films.profit==films.profit.min()].iloc[0]
print(answers['7']) # +

The Lone Ranger


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [124]:
answers['8'] = len(films[films.profit>0])
print(answers['8']) # +

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [125]:
answers['9'] = films.original_title\
    [(films.release_year==2008) & (films.revenue == films[films.release_year==2008].revenue.max())].iloc[0]
print(answers['9']) # +

The Dark Knight


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [126]:
answers['10'] = films.original_title\
    [((films.release_year>=2012) & (films.release_year<=2014)) \
     & (films.profit == films[(films.release_year>=2012) & (films.release_year<=2014)].profit.min())].iloc[0]
print(answers['10']) # +

The Lone Ranger


# 11. Какого жанра фильмов больше всего?

## Вариант 1. Подсчет статистики во вложенном цикле

In [127]:
genre_stats = Counter() # подсчет статистики жанров
for movie_genres in films.genres: # обход по фильмам
    for genre in movie_genres: # обход по списку жанров внутри фильма
        genre_stats[genre] += 1
answers['11'] = genre_stats.most_common(1)[0][0] # получить самый частый жанр
print(answers['11'])

Drama


## Вариант 2. Построение частотного словаря жанров

In [128]:
all_genres = films.genres.to_list() # построить полный список всех жанров
genres_list = [item for sublist in all_genres for item in sublist] # объединить списки жанров в плоский
genre_stats = {genre:genres_list.count(genre) for genre in genres_list} # построить частотный словарь жанров
answers['11'] = max(genre_stats, key=genre_stats.get) # получить самый частый жанр
print(answers['11'])

Drama


## Вариант 3. Денормализация списка фильмов по жанрам

In [129]:
films_by_genres = films.explode('genres', ignore_index=True) # денормализовать список фильмов по жанрам
genre_stats = films_by_genres.groupby('genres')['original_title'].count().reset_index() # сгруппировать статистику по жанрам
genre_stats.columns = ['genre', 'stats']
answers['11'] = genre_stats[genre_stats.stats == genre_stats.stats.max()].reset_index()['genre'][0] # форматировать ответ
print(answers['11'])

Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными?

In [130]:
# повтор денормализации осознанный, чтобы исключить влияние предыдущих заданий

films_by_genres = films.explode('genres', ignore_index=True) # денормализовать список фильмов по жанрам
profit_stats = films_by_genres[films_by_genres.profit>0].\
    groupby('genres')['original_title'].count().reset_index() # отобрать прибыльные фильмы
profit_stats.columns = ['genre', 'stats']
answers['12'] = profit_stats[profit_stats.stats == profit_stats.stats.max()].\
    reset_index()['genre'][0] # форматировать ответ
print(answers['12'])

Drama


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [131]:
films_by_dirs = films.explode('director', ignore_index=True) # денормализовать список фильмов по режисёрам
revenue_dirs = films_by_dirs.groupby('director')['revenue'].sum().reset_index()
answers['13'] = revenue_dirs[revenue_dirs.revenue == revenue_dirs.revenue.max()].reset_index()['director'][0]
print(answers['13'])

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [132]:
films_by_genres_dirs = films.explode('director', ignore_index=True).explode('genres', ignore_index=True) # денормализовать список фильмов по режисёрам и жанрам
action_dirs = films_by_genres_dirs[films_by_genres_dirs.genres == 'Action'].\
    groupby('director')['original_title'].count().reset_index()
action_dirs.columns = ['director', 'films_count']
answers['14'] = action_dirs[action_dirs.films_count == action_dirs.films_count.max()].\
    reset_index()['director'][0]
print(answers['14'])

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [133]:
films_by_actors = films.explode('cast', ignore_index=True) # денормализация по актёрам
revenue_actors = films_by_actors[films_by_actors.release_year==2012].groupby('cast')['revenue'].sum().reset_index()
answers['15'] = revenue_actors[revenue_actors.revenue == revenue_actors.revenue.max()].reset_index()['cast'][0]
print(answers['15'])

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [134]:
# Допущение: высокобюджетным считаем фильм с бюджетом выше среднего

films_by_actors = films[films.budget > films.budget.mean()].explode('cast', ignore_index=True) # денормализация по актёрам
answers['16'] = films_by_actors.groupby('cast')['original_title'].count().reset_index().\
    sort_values('original_title', ascending=False).reset_index()['cast'][0]
print(answers['16'])

Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [135]:
# константные значения
actor = 'Nicolas Cage' # актёр, по которому проводим анализ

films_by_actors = films.explode('cast', ignore_index=True) # денормализация по актёрам
films_by_actors_genres = films_by_actors[films_by_actors.cast == actor].explode('genres', ignore_index=True) # денормализация по жанрам
answers['17'] = films_by_actors_genres.groupby('genres')['original_title'].count().reset_index().\
    sort_values('original_title', ascending=False).reset_index()['genres'][0]
print(answers['17'])

Action


# 18. Самый убыточный фильм от Paramount Pictures

In [136]:
# константные значения
company = 'Paramount Pictures' # кинокомпания, по которой проводим анализ

films_by_studios = films.explode('production_companies', ignore_index=True) # денормализация по студиям
answers['18'] = films_by_studios[films_by_studios.production_companies == company].\
    sort_values('profit').reset_index()['original_title'][0]
print(answers['18'])

K-19: The Widowmaker


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [137]:
answers['19'] = films.groupby('release_year')['revenue'].sum().reset_index().\
    sort_values('revenue', ascending=False).reset_index()['release_year'][0]
print(answers['19'])

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [138]:
# константные значения
company = 'Warner' # группа кинокомпаний, входящих в студию WB

# Примечание: несмотря на указание в задаче студии "Warner Bros",
# поиск проводится с учетом студии "Warner Independent Pictures (WIP)",
# которая являлась подразделением WB согласно информации в википедии
# (https://en.wikipedia.org/wiki/Warner_Independent_Pictures)

films_by_studios = films.explode('production_companies', ignore_index=True) # денормализация по студиям
answers['20'] = films_by_studios[films_by_studios.production_companies.str.contains(company)].\
    groupby('release_year')['profit'].sum().reset_index().\
    sort_values('profit', ascending=False).reset_index()['release_year'][0]
print(answers['20'])

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [139]:
# Примечание: номер месяца преобразуется в текст для соответствия формату данных в ответах
answers['21'] = months[films.groupby('release_month')['original_title'].count().reset_index().\
    sort_values('original_title', ascending=False).reset_index()['release_month'][0]]
print(answers['21'])

Сентябрь


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [140]:
season = 'Лето' # Анализируемый сезон

answers['22'] = films[films['release_month'].isin(seasons[season])]['original_title'].count()
print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [141]:
season = 'Зима' # Анализируемый сезон

filmes_by_dirs = films[films['release_month'].isin(seasons[season])].explode('director', ignore_index=True) # денормализация по студиям
answers['23'] = filmes_by_dirs.groupby('director')['original_title'].count().reset_index().\
    sort_values('original_title', ascending=False).reset_index()['director'][0]
print(answers['23'])

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [142]:
# Примечание: имеется неоднозначность формулировки задачи - самые длинные названия могут быть в абсолютном выражении и по средним
# Допущение: самые длинные названия определяются по средней длине

films_by_studios = films.explode('production_companies', ignore_index=True) # денормализация по студиям
films_by_studios['title_len'] = [len(s) for s in films_by_studios['original_title']] # добавить столбец с длинами названий фильмов
answers['24'] = films_by_studios.groupby('production_companies')['title_len'].mean().reset_index().\
    sort_values('title_len', ascending=False).reset_index()['production_companies'][0]
print(answers['24'])

Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [143]:
films_by_studios = films.explode('production_companies', ignore_index=True) # денормализация по студиям
films_by_studios['title_len'] = [len(s) for s in films_by_studios['original_title']] # добавить столбец с длинами названий фильмов
films_by_studios['overview_words'] = [len(overview.split()) for overview in films_by_studios['overview']] # добавить столбец с длинами описаний
answers['25'] = films_by_studios.groupby('production_companies')['overview_words'].mean().reset_index().\
    sort_values('overview_words', ascending=False).reset_index()['production_companies'][0]
print(answers['25'])

Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [144]:
# параметры фильтрации
top_group = [['Inside Out', 'The Dark Knight', '12 Years a Slave'],
['BloodRayne', 'The Adventures of Rocky & Bullwinkle'],
['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down'],
['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1'],
['Upside Down', 'Inside Out', 'Iron Man']] # группы фильмов, которые нужно проверить на вхождение в топ-перцентиль
percentile = 1 # топ-перцентиль (1-100)
top_n = int(percentile * (len(films)/100)) # кол-во топовых компаний

top_list = films.sort_values('vote_average', ascending=False).head(top_n)['original_title'].to_list() # топ-список фильмов

top_intersection = [len(list(set(top_group[i]) & set(top_list))) for i in range(len(top_group))] # подсчет кол-ва вхождений в топ-список
max_intersection = max(top_intersection) # максимальное вхождение в топ-список
idx_intersection = top_intersection.index(max_intersection) # найти номер топ-группы, в которой максимум вхождений в топ-список
answers['26'] = top_group[idx_intersection]
print(answers['26'])

['Inside Out', 'The Dark Knight', '12 Years a Slave']


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?

**ВНИМАНИЕ! Выполнение кода занимает ~2,5 часа**

In [98]:
group_size = 2 # размер актёрской группы, по которой формируем статистику

cast_list = films['cast'].to_list() # построить полный список актёров
actors = pd.Series([item for sublist in cast_list for item in sublist]).unique() # список актёров без повторений
actors_groups = list(itertools.combinations(actors, group_size)) # построить все возможные актёрские пары

total_counter = len(actors_groups) # счетчик для оценки прогресса
percent_counter = total_counter/100
i = percent_counter # счетчик текущей операции
percentage = 0 # %% выполнения

film_counter = Counter() # счетчик совместных съёмок в фильмах
for actors_group in actors_groups:
    film_counter[actors_group] = \
        sum(list(map(lambda x: len(set(actors_group) & set(x)) == group_size, \
                     cast_list))) # просуммировать все совместные съёмки для группы актёров
    if i>= percent_counter:
        print(percentage,"% выполнено")
        percentage += 1
        i = 0 # сбросить временный счетчик на новую итерацию
    else:
        i += 1

0 % выполнено
1 % выполнено
2 % выполнено
3 % выполнено
4 % выполнено
5 % выполнено
6 % выполнено
7 % выполнено
8 % выполнено
9 % выполнено
10 % выполнено
11 % выполнено
12 % выполнено
13 % выполнено
14 % выполнено
15 % выполнено
16 % выполнено
17 % выполнено
18 % выполнено
19 % выполнено
20 % выполнено
21 % выполнено
22 % выполнено
23 % выполнено
24 % выполнено
25 % выполнено
26 % выполнено
27 % выполнено
28 % выполнено
29 % выполнено
30 % выполнено
31 % выполнено
32 % выполнено
33 % выполнено
34 % выполнено
35 % выполнено
36 % выполнено
37 % выполнено
38 % выполнено
39 % выполнено
40 % выполнено
41 % выполнено
42 % выполнено
43 % выполнено
44 % выполнено
45 % выполнено
46 % выполнено
47 % выполнено
48 % выполнено
49 % выполнено
50 % выполнено
51 % выполнено
52 % выполнено
53 % выполнено
54 % выполнено
55 % выполнено
56 % выполнено
57 % выполнено
58 % выполнено
59 % выполнено
60 % выполнено
61 % выполнено
62 % выполнено
63 % выполнено
64 % выполнено
65 % выполнено
66 % выполнено
67 % 

In [145]:
# параметры фильтрации
top_actors = [('Johnny Depp', 'Helena Bonham Carter'),
('Ben Stiller', 'Owen Wilson'),
('Vin Diesel', 'Paul Walker'),
('Adam Sandler', 'Kevin James'),
('Daniel Radcliffe', 'Rupert Grint')] # группы актёров, из которых нужно выбрать топ по совместным работам

joint_films = pd.DataFrame.from_dict(film_counter, orient='index').reset_index() # перевести counter -> df для фильтрации
joint_films.columns = ['actors', 'joint_count']
answers['27'] = list(list(\
    set(joint_films[joint_films.joint_count == joint_films.joint_count.max()]['actors'].to_list())\
    & set(top_actors)\
    )[0]) # построить пересечение списков топ-актёрских групп и проверяемых групп. Взять первую из них
print(answers['27'])

['Daniel Radcliffe', 'Rupert Grint']


# Submission

In [146]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 110,
 '5': 107,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
 '27': ['Daniel Radcliffe', 'Rupert Grint']}

In [147]:
# и убедиться что ни чего не пропустил)
len(answers)

27